In [1]:
import time
import argparse

import torch
import numpy as np

from utils import load_data

from gcn.train import get_model, test, normalize_adj

from defense import defense
from attack import attack

args = argparse.Namespace(dropout=0.5, epochs=100, 
                fastmode=False, hidden=16, lr=0.01, 
                seed=42, weight_decay=0.0005,
                use_gpu=True, verbose=False,
                defense_alpha=0.2, division_delta=1e-8,
                m=2, attack_delta=0.01, attack_delta_degree=False)
args.use_gpu = args.use_gpu and torch.cuda.is_available()
print(args)

np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.use_gpu:
    torch.cuda.manual_seed(args.seed)

idxs = {
    'train': torch.LongTensor(range(0, 270)), 
    'val': torch.LongTensor(range(270, 550)), 
    'test': torch.LongTensor(550+np.random.choice(1000, size=10, replace=False))
}

# Load data
adj, features, labels = load_data()
if args.use_gpu:
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    for key in idxs:
        idxs[key] = idxs[key].cuda()

args.attack_delta = int(args.attack_delta * adj.shape[0])


def train_attack_defense(adj, features, use_defense=False, use_attack=False):
    # load params
    # 1st defense removed
    #if use_defense:
    #    adj, features = defense(adj, features, args)

    t_total = time.time()
    model, _ = get_model(adj, features, labels, idxs, args)
    print("Optimization Finished!")
    # print("Total time elapsed: {:.4f}s".format(time.time() - t_total), flush=True)

    # Testing
    if use_attack:
        t_total = time.time()
        for i, node in enumerate(idxs['test'].cpu().numpy()):
            t_total = time.time()
            print(i, "of", list(idxs['test'].size()), end=" ")
            if args.attack_delta_degree:
                args.attack_delta = adj[node].sum()
            adj, features = attack(model, adj, features, labels, node, False, args) # 2nd defense removed
            print("time =", time.time() - t_total)
            
        print("Before Defense")
        adj_norm = normalize_adj(adj, args)
        _ = test(model, adj_norm, features, labels, idxs, args)
        
        if use_defense:
            adj, features = defense(adj, features, args)
            print("After Defense")
            adj_norm = normalize_adj(adj, args)
            _ = test(model, adj_norm, features, labels, idxs, args)
        # print("Total time elapsed: {:.4f}s".format(time.time() - t_total), flush=True)

print()
#train_attack_defense(adj, features, use_defense=False, use_attack=True)
#print()
train_attack_defense(adj, features, use_defense=True, use_attack=True)
print()
#args.attack_delta_degree = True
#train_attack_defense(adj, features, use_defense=False, use_attack=True)
#print()

Namespace(attack_delta=0.01, attack_delta_degree=False, defense_alpha=0.2, division_delta=1e-08, dropout=0.5, epochs=100, fastmode=False, hidden=16, lr=0.01, m=2, seed=42, use_gpu=True, verbose=False, weight_decay=0.0005)
Loading cora dataset...

Test set results: loss= 0.5024 accuracy= 0.8000
Optimization Finished!
0 of [10] time = 73.06095743179321
1 of [10] time = 73.39304256439209
2 of [10] time = 73.34412670135498
3 of [10] time = 73.33327913284302
4 of [10] time = 73.30122423171997
5 of [10] time = 73.85244822502136
6 of [10] time = 73.68070101737976
7 of [10] time = 73.80539774894714
8 of [10] time = 73.89022612571716
9 of [10] time = 73.40066409111023
Before Defense
Test set results: loss= 4.6718 accuracy= 0.0000
After Defense
Test set results: loss= 1.4563 accuracy= 0.6000

